###### 安裝必要的庫
!pip install pdf2image opencv-python-headless scikit-image tqdm
!apt-get install -y poppler-utils

In [ ]:
import os
import pickle
import csv
from pdf2image import convert_from_path
from tqdm import tqdm
from PIL import Image
import numpy as np
import io
from skimage.metrics import structural_similarity as compare_ssim
import json

# 設定資料夾的路徑
base_path = '/mnt/d/Esun'
source_folder = os.path.join(base_path, 'reference/finance_original_pdf')  # 包含股票代碼的資料夾
target_folder = os.path.join(base_path, 'reference/finance')  # 目標資料夾
log_file = 'finance_mapping.csv'  # CSV 記錄檔
source_images_folder = 'source_images'  # 儲存處理過的來源 PDF 檔案結果
company_names_file = os.path.join(base_path, 'company_names.json')  # 公司名稱映射檔案

# 加载 company_names.json 文件內容，該文件應包含 {"target_pdf_key": "company_name"} 格式的映射
with open(company_names_file, 'r') as f:
    company_names = json.load(f)

# 定義公司名稱與股票代碼的對應表
company_code_mapping = {
    "國巨股份有限公司": "2327",  # 國巨
    "華碩電腦股份有限公司": "2357",  # 華碩
    "台達電子工業股份有限公司": "2308",  # 台達電
    "統一企業股份有限公司": "1216",  # 統一
    "鴻海精密工業股份有限公司": "2317",  # 鴻海
    "研華股份有限公司": "2395",  # 研華
    "中國鋼鐵股份有限公司": "2002",  # 中鋼
    "和泰汽車股份有限公司": "2207",  # 和泰
    "聯華電子股份有限公司": "2303",  # 聯電
    "台灣化學纖維股份有限公司": "1326",  # 台化
    "中華電信股份有限公司": "2412",  # 中華電信
    "聯發科技股份有限公司": "2454",  # 聯發科
    "台灣積體電路製造股份有限公司": "2330",  # 台積電
    "光寶科技股份有限公司": "2301",  # 光寶科
    "長榮海運股份有限公司": "2603",  # 長榮
    "亞德客國際集團": "1590",  # 亞德客
    "瑞昱半導體股份有限公司": "2379",  # 瑞昱
    "台灣水泥股份有限公司": "1101",  # 台泥
    "智邦科技股份有限公司": "2345",  # 台泥
    "寶元數控股份有限公司": "4587"
    
    # 更多公司與代碼的對應關係可在此處手動加入...
}


# 定義函數：將 PDF 的所有頁轉換為灰階圖像，壓縮並儲存到內存中
def pdf_to_images(pdf_path):
    try:
        print(f"Processing {pdf_path} (all pages)...")
        images = convert_from_path(pdf_path)  # 處理所有頁
        image_data_list = []
        for image in images:
            # 將圖片轉換為灰階
            gray_image = image.convert('L')

            # 將圖片壓縮為 JPEG，並且不儲存到磁碟，而是儲存到內存
            img_byte_arr = io.BytesIO()
            gray_image.save(img_byte_arr, format='JPEG', quality=50)  # JPEG with compression
            image_data = img_byte_arr.getvalue()  # 獲取二進制數據
            image_data_list.append(image_data)
        return image_data_list
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return []

# 定義函數：將 PDF 的第一頁轉換為灰階圖像，壓縮並儲存到內存中
def pdf_to_first_image(pdf_path):
    try:
        print(f"Processing {pdf_path} (first page only)...")
        images = convert_from_path(pdf_path, first_page=1, last_page=1)  # 只處理第一頁
        image_data_list = []
        for image in images:
            # 將圖片轉換為灰階
            gray_image = image.convert('L')

            # 將圖片壓縮為 JPEG，並且不儲存到磁碟，而是儲存到內存
            img_byte_arr = io.BytesIO()
            gray_image.save(img_byte_arr, format='JPEG', quality=50)  # JPEG with compression
            image_data = img_byte_arr.getvalue()  # 獲取二進制數據
            image_data_list.append(image_data)
        return image_data_list
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return []

# 定義函數：比較兩張圖像是否相似（透過二進制數據）
def compare_images(img1_data, img2_data, threshold=0.99):
    try:
        # 使用 PIL 讀取二進制數據並轉換成圖像
        img1 = Image.open(io.BytesIO(img1_data))
        img2 = Image.open(io.BytesIO(img2_data))

        # 檢查兩個圖像的尺寸是否相同，如果不相同，將它們調整為相同尺寸
        if img1.size != img2.size:
            img2 = img2.resize(img1.size)

        # 將圖像轉換成 numpy 陣列進行 SSIM 比較
        img1_np = np.array(img1)
        img2_np = np.array(img2)

        # 計算結構相似度 (Structural Similarity Index, SSIM)
        score, _ = compare_ssim(img1_np, img2_np, full=True)
        return score >= threshold
    except Exception as e:
        print(f"Error comparing images: {e}")
        return False

# 創建 source_images 目錄，如果不存在
if not os.path.exists(source_images_folder):
    os.makedirs(source_images_folder)

# 檢查 CSV 記錄檔是否已經存在
if os.path.exists(log_file):
    with open(log_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        compared_targets = {row[0] for row in csvreader}  # 已經比對過的 target PDF 檔名集合
else:
    compared_targets = set()

# 如果 CSV 記錄檔不存在，創建並寫入標題
if not compared_targets:
    with open(log_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Target PDF', 'Stock Code', 'Source PDF', 'Page Number'])

# 讀取來源資料夾中包含兩層結構的所有 PDF，並將其圖像轉換並存儲到 pkl 檔案中
print("Processing source PDFs...")

for stock_folder in tqdm(os.listdir(source_folder), desc="Processing source folders"):
    stock_folder_path = os.path.join(source_folder, stock_folder)

    if os.path.isdir(stock_folder_path):  # 檢查是否是股票代碼資料夾
        stock_pkl_path = os.path.join(source_images_folder, f'{stock_folder}.pkl')

        # 如果該股票代碼的 pkl 已經存在，則跳過
        if os.path.exists(stock_pkl_path):
            print(f"Source PDFs [{stock_folder}] already processed, skipping...")
            continue

        source_images = {}

        for pdf_file in os.listdir(stock_folder_path):
            if pdf_file.endswith('.pdf'):
                source_path = os.path.join(stock_folder_path, pdf_file)
                try:
                    # 將所有頁面轉為圖片
                    image_data_list = pdf_to_images(source_path)
                    source_images[pdf_file] = image_data_list
                    print(f"Successfully processed {source_path}")
                except Exception as e:
                    print(f"Failed to process {source_path}: {e}")

        # 保存每個 PDF 圖像數據到對應的 pkl 檔案
        with open(stock_pkl_path, 'wb') as f:
            pickle.dump(source_images, f)
        print(f"Saved images for stock code {stock_folder} to {stock_pkl_path}")

print("Source PDFs processed. Starting target PDF comparison...")

# 定義一個緩存字典，用於存儲已讀取的 source_images，避免重複讀取
source_images_cache = {}

# 讀取目標資料夾中的所有 PDF，並只比對第一頁
for target_pdf in tqdm(os.listdir(target_folder), desc="Processing target PDFs"):
    if target_pdf.endswith('.pdf'):
        if target_pdf in compared_targets:
            print(f"{target_pdf} already compared, skipping...")
            continue

        target_pdf_key = os.path.splitext(target_pdf)[0]  # 去掉副檔名

        if target_pdf_key in company_names:  # 如果公司名稱在映射檔案中
            company_name = company_names[target_pdf_key]
            if company_name in company_code_mapping:  # 如果公司名稱有對應股票代碼
                stock_code = company_code_mapping[company_name]

                # 檢查該目標 PDF 是否已經比對過
                if target_pdf in compared_targets:
                    print(f"{target_pdf} already compared, skipping...")
                    continue

                target_path = os.path.join(target_folder, target_pdf)
                target_images = pdf_to_first_image(target_path)  # 只處理第一頁

                # 從緩存中讀取來源 PDF，如果緩存中沒有，則從 pkl 檔案讀取
                if stock_code not in source_images_cache:
                    stock_pkl_path = os.path.join(source_images_folder, f'{stock_code}.pkl')
                    if os.path.exists(stock_pkl_path):
                        with open(stock_pkl_path, 'rb') as f:
                            source_images_cache[stock_code] = pickle.load(f)
                    else:
                        print(f"No source images found for stock code {stock_code}")
                        continue

                source_images = source_images_cache[stock_code]

                with open(log_file, 'a', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    found = False
                    print(f"Comparing first page of {target_pdf}...")

                    # 比對目標 PDF 第一頁與來源 PDF 所有頁面
                    for source_pdf, source_image_data_list in tqdm(source_images.items(), desc=f"Comparing {target_pdf} with source PDFs"):
                        for source_page_num, source_image_data in enumerate(source_image_data_list):
                            if compare_images(target_images[0], source_image_data):  # 比較圖像
                                csvwriter.writerow([target_pdf, stock_code, source_pdf, source_page_num + 1])
                                print(f"Match found: {target_pdf} first page matches {source_pdf} page {source_page_num + 1}")
                                found = True
                                break
                        if found:
                            break
                    if not found:
                        csvwriter.writerow([target_pdf, 'N/A', 'N/A', 'N/A'])  # 如果找不到匹配頁面
                        print(f"No match found for {target_pdf} first page")
            else:
                continue  # 沒有對應股票代碼則跳過
        else:
            print(f"Target PDF {target_pdf_key} not found in company_names.json, skipping...")

print("Done")

In [ ]:
import os
import pickle
import csv
from pdf2image import convert_from_path
from tqdm import tqdm
from PIL import Image
import numpy as np
import io
from skimage.metrics import structural_similarity as compare_ssim

# 設定資料夾的路徑
base_path = '/mnt/d/Esun'
source_folder = os.path.join(base_path, 'reference/finance_original_pdf')
target_folder = os.path.join(base_path, 'reference/finance')
log_file = 'finance_mapping.csv'
source_images_folder = 'source_images'

# 將 PDF 轉換成灰階圖片的功能
def pdf_to_images(pdf_path):
    print(f"處理 {pdf_path} (全部頁面)...")
    images = convert_from_path(pdf_path)
    image_data_list = []
    for image in images:
        gray_image = image.convert('L')
        img_byte_arr = io.BytesIO()
        gray_image.save(img_byte_arr, format='JPEG', quality=50)
        image_data = img_byte_arr.getvalue()
        image_data_list.append(image_data)
    return image_data_list

def pdf_to_first_image(pdf_path):
    print(f"處理 {pdf_path} (僅首頁)...")
    images = convert_from_path(pdf_path, first_page=1, last_page=1)
    image_data_list = []
    for image in images:
        gray_image = image.convert('L')
        img_byte_arr = io.BytesIO()
        gray_image.save(img_byte_arr, format='JPEG', quality=50)
        image_data = img_byte_arr.getvalue()
        image_data_list.append(image_data)
    return image_data_list

def compare_images(img1_data, img2_data, threshold=0.99):
    img1 = Image.open(io.BytesIO(img1_data))
    img2 = Image.open(io.BytesIO(img2_data))
    if img1.size != img2.size:
        img2 = img2.resize(img1.size)
    img1_np = np.array(img1)
    img2_np = np.array(img2)
    score, _ = compare_ssim(img1_np, img2_np, full=True)
    return score >= threshold

# 建立資料夾
if not os.path.exists(source_images_folder):
    os.makedirs(source_images_folder)

# 讀取比對記錄
compared_targets = set()
if os.path.exists(log_file):
    with open(log_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            if row[1] != 'N/A':  # 忽略沒有找到匹配的記錄
                compared_targets.add(row[0])

# 創建 CSV 檔案如果不存在
if not compared_targets:
    with open(log_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Target PDF', 'Stock Code', 'Source PDF', 'Page Number'])

# 緩存來源圖像
source_images_cache = {}

# 處理所有 pkl 檔案
for pkl_file in os.listdir(source_images_folder):
    pkl_path = os.path.join(source_images_folder, pkl_file)
    stock_code = os.path.splitext(pkl_file)[0]
    with open(pkl_path, 'rb') as f:
        source_images_cache[stock_code] = pickle.load(f)

# 比對目標 PDF 與來源 PDF
for target_pdf in tqdm(os.listdir(target_folder), desc="處理目標 PDF"):
    if target_pdf.endswith('.pdf'):
        target_pdf_key = os.path.splitext(target_pdf)[0]
        if target_pdf in compared_targets:
            continue

        target_path = os.path.join(target_folder, target_pdf)
        target_images = pdf_to_first_image(target_path)

        with open(log_file, 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            found = False
            print(f"比對 {target_pdf} 的首頁...")

            for stock_code, source_images in source_images_cache.items():
                for source_pdf, source_image_data_list in tqdm(source_images.items(), desc=f"與 {target_pdf} 比對的來源 PDF"):
                    for source_page_num, source_image_data in enumerate(source_image_data_list):
                        if compare_images(target_images[0], source_image_data):
                            csvwriter.writerow([target_pdf, stock_code, source_pdf, source_page_num + 1])
                            print(f"找到匹配：{target_pdf} 首頁與 {source_pdf} 第 {source_page_num + 1} 頁相符")
                            found = True
                            break
                    if found:
                        break
                if found:
                    break

            if not found:
                csvwriter.writerow([target_pdf, 'N/A', 'N/A', 'N/A'])
                print(f"未找到匹配：{target_pdf} 的首頁")

print("處理完畢")


處理目標 PDF:   0%|          | 0/1036 [00:00<?, ?it/s]

處理 /mnt/d/Esun/reference/finance/493.pdf (僅首頁)...
比對 493.pdf 的首頁...



與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [09:08<00:00, 68.51s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [05:24<00:00, 40.62s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [03:51<00:00, 28.97s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 4/4 [01:45<00:00, 26.27s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [06:58<00:00, 52.36s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [08:06<00:00, 60.79s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [05:38<00:00, 42.37s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [06:51<00:00, 51.47s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [12:03<00:00, 90.47s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [04:05<00:00, 30.73s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [07:14<00:00, 54.25s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 6/6 [03:08<00:00, 31.48s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [07:01<00:00, 52.63s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [06:17<00:00, 47.18s/it]

與 493.pdf 比對的來源 PDF: 100%|██████████| 8/8 [07:4

找到匹配：493.pdf 首頁與 202303_2357_AI1_20241018_101616.pdf 第 56 頁相符
處理 /mnt/d/Esun/reference/finance/497.pdf (僅首頁)...
比對 497.pdf 的首頁...



與 497.pdf 比對的來源 PDF: 100%|██████████| 8/8 [09:04<00:00, 68.04s/it]

與 497.pdf 比對的來源 PDF: 100%|██████████| 8/8 [05:19<00:00, 39.92s/it]

與 497.pdf 比對的來源 PDF: 100%|██████████| 8/8 [04:01<00:00, 30.14s/it]

與 497.pdf 比對的來源 PDF: 100%|██████████| 4/4 [01:55<00:00, 28.95s/it]

與 497.pdf 比對的來源 PDF: 100%|██████████| 8/8 [07:47<00:00, 58.50s/it]

與 497.pdf 比對的來源 PDF: 100%|██████████| 8/8 [09:08<00:00, 68.54s/it]

處理目標 PDF:  46%|████▌     | 478/1036 [2:39:24<3:24:05, 21.94s/it]

...


與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [10:45<00:00, 80.67s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [06:11<00:00, 46.46s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [04:28<00:00, 33.61s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 4/4 [01:56<00:00, 29.18s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [08:02<00:00, 60.33s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [10:01<00:00, 75.15s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [06:59<00:00, 52.38s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [08:34<00:00, 64.27s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [15:09<00:00, 113.65s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [05:12<00:00, 39.02s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [08:57<00:00, 67.15s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 6/6 [03:53<00:00, 38.94s/it]

與 581.pdf 比對的來源 PDF: 100%|██████████| 8/8 [08:58<00:00, 67.31s/it]

處理目標 PDF:  55%|█████▌    | 572/1036 [5:53:38<7:46:40, 60.35s/it]

找到匹配：581.pdf 首頁與 202301_2412_AI1_20241018_230459.pdf 第 17 頁相符
處理 /mnt/d/Esun/reference/finance/623.pdf (僅首頁)...
比對 623.pdf 的首頁...



與 623.pdf 比對的來源 PDF: 100%|██████████| 8/8 [11:49<00:00, 88.68s/it]

與 623.pdf 比對的來源 PDF: 100%|██████████| 8/8 [06:58<00:00, 52.32s/it]

處理目標 PDF:  60%|█████▉    | 619/1036 [6:16:18<6:05:21, 52.57s/it]

找到匹配：623.pdf 首頁與 202303_1590_AI1_20241019_100446.pdf 第 12 頁相符
處理 /mnt/d/Esun/reference/finance/637.pdf (僅首頁)...
比對 637.pdf 的首頁...



與 637.pdf 比對的來源 PDF: 100%|██████████| 8/8 [11:31<00:00, 86.47s/it]

與 637.pdf 比對的來源 PDF:   0%|          | 0/8 [00:00<?, ?it/s]